In [1]:
import qiskit
qiskit.__version__

'1.3.2'

In [2]:
import numpy as np
import re
import random
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.primitives import StatevectorSampler
import matplotlib.pyplot as plt
from qiskit.visualization import plot_histogram

In [3]:
numberOfSinglets = 16 # let alice and bob use 16 singlets for the key , bigger key better numeric results

Now we need to prepapre to measure the state using three different basis state each for both alice and bob. A1, A2, A3 are for alice and B1,B2,B3 for bob

In [4]:
def create_measurement_circuit(qr, cr, alice_choice, bob_choice):
    circuit = QuantumCircuit(qr, cr)
    
    # Implement Alice's measurement based on random choice
    if alice_choice == 1:  # measureA1
        circuit.h(qr[0])
    elif alice_choice == 2:  # measureA2
        circuit.s(qr[0])
        circuit.h(qr[0])
        circuit.t(qr[0])
        circuit.h(qr[0])
    # alice_choice == 3 (measureA3) only needs measurement
    
    # Implement Bob's measurement based on random choice
    if bob_choice == 1:  # measureB1
        circuit.s(qr[1])
        circuit.h(qr[1])
        circuit.t(qr[1])
        circuit.h(qr[1])
    elif bob_choice == 3:  # measureB3
        circuit.s(qr[1])
        circuit.h(qr[1])
        circuit.tdg(qr[1])
        circuit.h(qr[1])
    # bob_choice == 2 (measureB2) only needs measurement
    
    # Add measurements for both qubits
    circuit.measure(qr[0], cr[0])
    circuit.measure(qr[1], cr[1])
    
    return circuit

In [5]:
aliceMeasurementChoices = [random.randint(1, 3) for i in range(numberOfSinglets)]
bobMeasurementChoices = [random.randint(1, 3) for i in range(numberOfSinglets)]

In [6]:
circuits = [] # combined circuit consisting of singlet and alice and bob's measurement in one of the three basis each

for i in range(numberOfSinglets):
    # Create the complete circuit
    qr = QuantumRegister(2, name="qr")
    cr = ClassicalRegister(4, name="cr")
    circuitName = str(i) + ':A' + str(aliceMeasurementChoices[i]) + '_B' + str(bobMeasurementChoices[i]) #step 1: charlie prepares the singlet state
    circuit = QuantumCircuit(qr, cr, name=circuitName)
    circuit.x(qr[0])
    circuit.x(qr[1])
    circuit.h(qr[0])
    circuit.cx(qr[0],qr[1])
    # alice and bob's measurement according to basis
    measurement_circuit = create_measurement_circuit(
        qr, cr, 
        aliceMeasurementChoices[i], 
        bobMeasurementChoices[i]
    )
    
    
    circuit.compose(measurement_circuit, inplace=True)
    # Add circuit to the list
    circuits.append((circuit))

In [7]:
sampler = StatevectorSampler()
job = sampler.run(circuits,shots=1)
result = job.result()
counts = result[0].data.cr.get_counts()
counts

{'0001': 1}

In [8]:
list(counts.keys())[0]

'0001'

In [9]:
abPatterns = [
    re.compile('..00$'), # search for the '..00' output (Alice obtained -1 and Bob obtained -1)
    re.compile('..01$'), # search for the '..01' output
    re.compile('..10$'), # search for the '..10' output (Alice obtained -1 and Bob obtained 1)
    re.compile('..11$')  # search for the '..11' output
]

In [10]:
aliceResults = [] # Alice's results (string a)
bobResults = [] # Bob's results (string a')

for i in range(numberOfSinglets):

    res = list(result[i].data.cr.get_counts().keys())[0] 
    
    if abPatterns[0].search(res): 
        aliceResults.append(-1) 
        bobResults.append(-1) 
    if abPatterns[1].search(res):
        aliceResults.append(1)
        bobResults.append(-1)
    if abPatterns[2].search(res): 
        aliceResults.append(-1) 
        bobResults.append(1) 
    if abPatterns[3].search(res): 
        aliceResults.append(1)
        bobResults.append(1)

In [11]:
aliceKey = [] # Alice's key string k
bobKey = [] # Bob's key string k'

# comparing the stings with measurement choices
for i in range(numberOfSinglets):
   
    if (aliceMeasurementChoices[i] == 2 and bobMeasurementChoices[i] == 1) or (aliceMeasurementChoices[i] == 3 and bobMeasurementChoices[i] == 2):
        aliceKey.append(aliceResults[i]) 
        bobKey.append(- bobResults[i])
        
keyLength = len(aliceKey)

In [12]:
abKeyMismatches = 0 
for j in range(keyLength):
    if aliceKey[j] != bobKey[j]:
        abKeyMismatches += 1

In [13]:
# function that calculates CHSH correlation value
def chsh_corr(result):
    
    # lists with the counts of measurement results
    # each element represents the number of (-1,-1), (-1,1), (1,-1) and (1,1) results respectively
    countA1B1 = [0, 0, 0, 0] # XW observable
    countA1B3 = [0, 0, 0, 0] # XV observable
    countA3B1 = [0, 0, 0, 0] # ZW observable
    countA3B3 = [0, 0, 0, 0] # ZV observable

    for i in range(numberOfSinglets):

        res = list(result[i].data.cr.get_counts().keys())[0]

        # if the spins of the qubits of the i-th singlet were projected onto the a_1/b_1 directions
        if (aliceMeasurementChoices[i] == 1 and bobMeasurementChoices[i] == 1):
            for j in range(4):
                if abPatterns[j].search(res):
                    countA1B1[j] += 1

        if (aliceMeasurementChoices[i] == 1 and bobMeasurementChoices[i] == 3):
            for j in range(4):
                if abPatterns[j].search(res):
                    countA1B3[j] += 1

        if (aliceMeasurementChoices[i] == 3 and bobMeasurementChoices[i] == 1):
            for j in range(4):
                if abPatterns[j].search(res):
                    countA3B1[j] += 1
                    
        # if the spins of the qubits of the i-th singlet were projected onto the a_3/b_3 directions
        if (aliceMeasurementChoices[i] == 3 and bobMeasurementChoices[i] == 3):
            for j in range(4):
                if abPatterns[j].search(res):
                    countA3B3[j] += 1
                    
    # number of the results obtained from the measurements in a particular basis
    total11 = sum(countA1B1)
    total13 = sum(countA1B3)
    total31 = sum(countA3B1)
    total33 = sum(countA3B3)      
                    
    # expectation values of XW, XV, ZW and ZV observables (2)
    expect11 = (countA1B1[0] - countA1B1[1] - countA1B1[2] + countA1B1[3])/total11 # -1/sqrt(2)
    expect13 = (countA1B3[0] - countA1B3[1] - countA1B3[2] + countA1B3[3])/total13 # 1/sqrt(2)
    expect31 = (countA3B1[0] - countA3B1[1] - countA3B1[2] + countA3B1[3])/total31 # -1/sqrt(2)
    expect33 = (countA3B3[0] - countA3B3[1] - countA3B3[2] + countA3B3[3])/total33 # -1/sqrt(2) 
    
    corr = expect11 - expect13 + expect31 + expect33 # calculate the CHSC correlation value (3)
    
    return corr

In [14]:
corr = chsh_corr(result) # CHSH correlation value

# CHSH inequality test -2root(2) = -2.828
print('CHSH correlation value: ' + str(round(corr, 3)))

# Keys
print('Length of the key: ' + str(keyLength))
print('Number of mismatching bits: ' + str(abKeyMismatches) + '\n')

CHSH correlation value: -3.333
Length of the key: 1
Number of mismatching bits: 0



# Now if eve is eavesdropping

In [15]:
def create_measurement_circuit(qr, cr, alice_choice, bob_choice, eve_choices):
    circuit = QuantumCircuit(qr, cr)
    
    if eve_choices[0] == 2:  # measureEA2
        circuit.s(qr[0])
        circuit.h(qr[0])
        circuit.t(qr[0])
        circuit.h(qr[0])
        circuit.measure(qr[0], cr[2])
    elif eve_choices[0] == 3:  # measureEA3
        circuit.measure(qr[0], cr[2])
        
    
    if eve_choices[1] == 1:  # measureEB1
        circuit.s(qr[1])
        circuit.h(qr[1])
        circuit.t(qr[1])
        circuit.h(qr[1])
        circuit.measure(qr[1], cr[3])
    elif eve_choices[1] == 2:  # measureEB2
        circuit.measure(qr[1], cr[3])
    
   # Implement Alice's measurement based on random choice
    if alice_choice == 1:  # measureA1
        circuit.h(qr[0])
    elif alice_choice == 2:  # measureA2
        circuit.s(qr[0])
        circuit.h(qr[0])
        circuit.t(qr[0])
        circuit.h(qr[0])
    # alice_choice == 3 (measureA3) only needs measurement
    
    # Implement Bob's measurement based on random choice
    if bob_choice == 1:  # measureB1
        circuit.s(qr[1])
        circuit.h(qr[1])
        circuit.t(qr[1])
        circuit.h(qr[1])
    elif bob_choice == 3:  # measureB3
        circuit.s(qr[1])
        circuit.h(qr[1])
        circuit.tdg(qr[1])
        circuit.h(qr[1])
    # bob_choice == 2 (measureB2) only needs measurement
    
    # Add measurements for both qubits
    circuit.measure(qr[0], cr[0])
    circuit.measure(qr[1], cr[1])
    
    return circuit

In [16]:
aliceMeasurementChoices = [random.randint(1, 3) for _ in range(numberOfSinglets)]
bobMeasurementChoices = [random.randint(1, 3) for _ in range(numberOfSinglets)]

# Generate Eve's measurement choices
eveMeasurementChoices = []
for _ in range(numberOfSinglets):
    if random.uniform(0, 1) <= 0.5:  # WW measurement
        eveMeasurementChoices.append([2, 1])
    else:  # ZZ measurement
        eveMeasurementChoices.append([3, 2])

In [17]:
circuits = []
for i in range(numberOfSinglets):
    # Create the complete circuit
    qr = QuantumRegister(2, name="qr")
    cr = ClassicalRegister(4, name="cr")
    circuitname = circuitName = str(i) + ':A' + str(aliceMeasurementChoices[i]) + '_E' + str(eveMeasurementChoices[i]) + '_B' + str(bobMeasurementChoices[i]) 
    circuit = QuantumCircuit(qr, cr, name=circuitName)
    circuit.x(qr[0])
    circuit.x(qr[1])
    circuit.h(qr[0])
    circuit.cx(qr[0],qr[1])
    measurement_circuit = create_measurement_circuit(
        qr, cr,
        aliceMeasurementChoices[i],
        bobMeasurementChoices[i],
        eveMeasurementChoices[i]
    )
    
    # Combine singlet and measurement circuits
    circuit.compose(measurement_circuit, inplace=True)
    circuits.append(circuit)

In [18]:
from qiskit_aer import AerSimulator

# Can't use sampler due to mid ciruit measurements
simulator = AerSimulator()
job = simulator.run(circuits, shots=1)
results = job.result()
counts = results.get_counts()

In [19]:
counts

[{'1011': 1},
 {'1010': 1},
 {'1010': 1},
 {'1010': 1},
 {'1000': 1},
 {'1010': 1},
 {'1010': 1},
 {'0101': 1},
 {'0101': 1},
 {'1010': 1},
 {'1010': 1},
 {'1010': 1},
 {'0101': 1},
 {'0101': 1},
 {'1010': 1},
 {'1011': 1}]

In [20]:
ePatterns = [
    re.compile('00..$'), # search for the '00..' result (Eve obtained the results -1 and -1 for Alice's and Bob's qubits)
    re.compile('01..$'), # search for the '01..' result (Eve obtained the results 1 and -1 for Alice's and Bob's qubits)
    re.compile('10..$'),
    re.compile('11..$')  
]

In [21]:
aliceResults = [] # Alice's results (string a)
bobResults = [] # Bob's results (string a')

# list of Eve's measurement results
# the elements in the 1-st column are the results obtaned from the measurements of Alice's qubits
# the elements in the 2-nd column are the results obtaned from the measurements of Bob's qubits
eveResults = [] 

# recording the measurement results
for j in range(numberOfSinglets):
    
    res = list(results.get_counts(circuits[j]).keys())[0] 
    
    # Alice and Bob
    if abPatterns[0].search(res): 
        aliceResults.append(-1) 
        bobResults.append(-1) 
    if abPatterns[1].search(res):
        aliceResults.append(1)
        bobResults.append(-1)
    if abPatterns[2].search(res): 
        aliceResults.append(-1) 
        bobResults.append(1) 
    if abPatterns[3].search(res): 
        aliceResults.append(1)
        bobResults.append(1)

    # Eve
    if ePatterns[0].search(res):
        eveResults.append([-1, -1]) 
    if ePatterns[1].search(res):
        eveResults.append([1, -1])
    if ePatterns[2].search(res):
        eveResults.append([-1, 1])
    if ePatterns[3].search(res):
        eveResults.append([1, 1])

In [22]:
aliceKey = [] # Alice's key string a
bobKey = [] # Bob's key string a'
eveKeys = [] # Eve's keys; the 1-st column is the key of Alice, and the 2-nd is the key of Bob

# comparing the strings with measurement choices (b and b')
for j in range(numberOfSinglets):
    # if Alice and Bob measured the spin projections onto the a_2/b_1 or a_3/b_2 directions
    if (aliceMeasurementChoices[j] == 2 and bobMeasurementChoices[j] == 1) or (aliceMeasurementChoices[j] == 3 and bobMeasurementChoices[j] == 2):  
        aliceKey.append(aliceResults[j]) 
        bobKey.append(-bobResults[j]) 
        eveKeys.append([eveResults[j][0], -eveResults[j][1]]) 
keyLength = len(aliceKey) 

In [23]:
abKeyMismatches = 0 # number of mismatching bits in the keys of Alice and Bob
eaKeyMismatches = 0 # number of mismatching bits in the keys of Eve and Alice
ebKeyMismatches = 0 # number of mismatching bits in the keys of Eve and Bob

for j in range(keyLength):
    if aliceKey[j] != bobKey[j]: 
        abKeyMismatches += 1
    if eveKeys[j][0] != aliceKey[j]:
        eaKeyMismatches += 1
    if eveKeys[j][1] != bobKey[j]:
        ebKeyMismatches += 1

In [24]:
eaKnowledge = (keyLength - eaKeyMismatches)/keyLength # Eve's knowledge of Bob's key
ebKnowledge = (keyLength - ebKeyMismatches)/keyLength # Eve's knowledge of Alice's key

In [25]:
corr = chsh_corr(result)
corr

2.0

In [26]:
# CHSH inequality test
print('CHSH correlation value: ' + str(round(corr, 3)) + '\n')

# Keys
print('Length of the key: ' + str(keyLength))
print('Number of mismatching bits: ' + str(abKeyMismatches) + '\n')

print('Eve\'s knowledge of Alice\'s key: ' + str(round(eaKnowledge * 100, 2)) + ' %')
print('Eve\'s knowledge of Bob\'s key: ' + str(round(ebKnowledge * 100, 2)) + ' %')

CHSH correlation value: 2.0

Length of the key: 3
Number of mismatching bits: 0

Eve's knowledge of Alice's key: 100.0 %
Eve's knowledge of Bob's key: 100.0 %


# References : 
https://github.com/qiskit-community/qiskit-community-tutorials/blob/6de54e7033edc4233142caecda257ed72a6735f5/awards/teach_me_qiskit_2018/e91_qkd/e91_quantum_key_distribution_protocol.ipynb